In [1]:
import sqlite3
import numpy as np
import time
from matplotlib import pyplot as plt

In [2]:
floor28_db = 'BlackBox/BlackBox_2&8_floor.db'
floor28_conn = sqlite3.connect(floor28_db)
cur28 = floor28_conn.cursor()

In [3]:
floor67_db = 'BlackBox/BlackBox_6&7_floor.db'
floor67_conn = sqlite3.connect(floor67_db)
cur67 = floor67_conn.cursor()

In [4]:
def get_fingerprint_list(rows, key=lambda x : (x['rss'], x['bssid']), reverse=True):
    fp_dict = dict()
    prev_time = -1e8
    
    for row in rows:
        element = {
            'timeMicros': row[1],
            'ssid': row[2],
            'bssid': row[3],
            'freq': row[4],
            'rss': row[5]
        }
        wifi_time = row[6]
        
        try:
            fp_dict[wifi_time].append(element)
        except KeyError:
            fp_dict[wifi_time] = [element]
    
    fp_list = [{'logTime': t, 'data': sorted(fp_dict[t], key=key, reverse=reverse)} for t in sorted(fp_dict)]
    return fp_list

In [6]:
#6th, 7th floor barometer
cur67.execute("select * from Rss where id >= 1064603 and id <= 1065635")
rows_6th_floor = cur67.fetchall()

cur67.execute("select * from Rss where id >= 1053927 and id <= 1063057")
rows_7_1_floor = cur67.fetchall()

cur67.execute("select * from Rss where id >= 1063058 and id <= 1064279")
rows_7_2_floor = cur67.fetchall()


#2nd, 8th floor fingerprint
cur28.execute("select * from Rss where id >= 280 and id < 2436")
rows_8th_floor = cur28.fetchall()

cur28.execute("select * from Rss where id >= 2436")
rows_2nd_floor = cur28.fetchall()

In [31]:
fp_list_confirmed = get_fingerprint_list(rows_8th_floor)
fp_list_user = get_fingerprint_list(rows_6th_floor)

In [25]:
def print_common_fp(data, bssid_list):
    for row in data:
        if row['bssid'] in bssid_list:
            print((row['ssid'], row['bssid'], row['rss']))
    
    print()

In [32]:
# Jacard Coefficient == n(A∩B) / n(A∪B)
# Jacard Coefficient Threshold = _J
# Count Threshold = _C
_J, _C = 0.5, 0.8
count = 0

for confirmed, user in zip(fp_list_confirmed, fp_list_user):
    confirmed_bssids = [data['bssid'] for data in confirmed['data']]
    user_bssids = [data['bssid'] for data in user['data']]
    
    intersection = list(set(confirmed_bssids) & set(user_bssids))
    union = list(set(confirmed_bssids) | set(user_bssids))
    
    if not union or not intersection:
        continue
    
    jac_coeff = len(intersection) / len(union)
    
    ######################
    print('-' * 80)
    print('len(union): %d, len(intersection): %d, jac_coeff: %.3f\n' 
          % (len(union), len(intersection), len(intersection) / len(union)), )
    print('intersection(confirmed):')
    print_common_fp(confirmed['data'], intersection)
    print('intersection(user):')
    print_common_fp(user['data'], intersection)
    ######################
    
    if jac_coeff >= _J:
        count += 1

if count / min(len(fp_list_confirmed), len(fp_list_user)) > _C:
    print('Same Floor')
else:
    print('Different Floor')

--------------------------------------------------------------------------------
len(union): 97, len(intersection): 10, jac_coeff: 0.103

intersection(confirmed):
('Lapras5G-guest', '62:38:e0:bb:ee:02', -64)
('iDBLab-2.4GHz', '78:54:2e:4c:f1:50', -67)
('mcwire', '08:10:77:a8:8c:e9', -72)
('WASHERE1', 'b0:c7:45:76:44:2a', -73)
('MLILAB824_2', '8a:36:6c:c9:4e:70', -80)
('WASHERE2', 'b0:c7:45:76:44:2b', -82)
('egocoord', '72:5d:cc:4b:ae:a4', -87)
('HCIL', '6c:70:9f:e0:92:7f', -88)
('hcil_vr4t5G', '70:5d:cc:78:5b:28', -89)
('eduroam', '00:24:6c:7d:f6:e1', -89)

intersection(user):
('WASHERE1', 'b0:c7:45:76:44:2a', -68)
('iDBLab-2.4GHz', '78:54:2e:4c:f1:50', -71)
('egocoord', '72:5d:cc:4b:ae:a4', -72)
('hcil_vr4t5G', '70:5d:cc:78:5b:28', -72)
('Lapras5G-guest', '62:38:e0:bb:ee:02', -73)
('MLILAB824_2', '8a:36:6c:c9:4e:70', -76)
('HCIL', '6c:70:9f:e0:92:7f', -76)
('eduroam', '00:24:6c:7d:f6:e1', -76)
('mcwire', '08:10:77:a8:8c:e9', -78)
('WASHERE2', 'b0:c7:45:76:44:2b', -81)

---------------

In [155]:
floor28_conn.close()
floor67_conn.close()